In [1]:
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
best = pd.read_csv('best.csv', index_col=0)

In [3]:
concerned_Rs = [1, 3, 6, 9]
concerned_rhos = [0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8]
limits = {1:22, 2:22, 3:22, 4:13, 5:7, 6:5, 7:4, 8:3, 9:3, 10:2}

In [4]:
def get_percentiles(R, rho, percentile):
    simulation = pd.read_csv(f'sweep-expon/expon-R={R}-rho={rho}.csv', index_col=0)
    simulation_percentile = list(simulation[simulation['cdf'] > percentile]['sojourn_time'])[0]
    method = list(best[(best['R'] == R) & (best['rho'] == rho)]['best'])[0]
    lim = limits[R]
    approx_cdf = pd.read_csv(f'sweep/method{method}/mc_limit={lim}-infty=130-R={R}-rho={rho}.csv', index_col=0)
    approx_percentile = list(approx_cdf[approx_cdf['cdf'] > percentile]['sojourn_time'])[0]
    return simulation_percentile, approx_percentile

In [5]:
def get_percentile_error(R, rho, percentile):
    simulation_percentile, approx_percentile = get_percentiles(R, rho, percentile)
    return abs(simulation_percentile - approx_percentile) / simulation_percentile

In [6]:
def make_table_percentiles(concerned_Rs, concerned_rhos, percentile):
    table = pd.DataFrame({
        f'R={R}': [get_percentiles(R=R, rho=rho, percentile=percentile)[0] for rho in concerned_rhos] for R in concerned_Rs
    }, index=[f"$\rho = {rho}$" for rho in concerned_rhos]).round(2)
    return table

In [7]:
def make_table_errors(concerned_Rs, concerned_rhos, percentile):
    table = pd.DataFrame({
        f'R={R}': [get_percentile_error(R=R, rho=rho, percentile=percentile) for rho in concerned_rhos] for R in concerned_Rs
    }, index=[f"$\rho = {rho}$" for rho in concerned_rhos]).round(2)
    return table

# 99th Percentile

In [8]:
make_table_errors(concerned_Rs, concerned_rhos, 0.99)

R=1   R=3   R=6   R=9
$\rho = 0.2$  0.01  0.19  0.22  0.22
$\rho = 0.3$  0.03  0.22  0.30  0.31
$\rho = 0.4$  0.03  0.24  0.36  0.39
$\rho = 0.5$  0.01  0.23  0.38  0.44
$\rho = 0.6$  0.02  0.19  0.07  0.09
$\rho = 0.7$  0.03  0.07  0.03  0.01
$\rho = 0.8$  0.00  0.06  0.05  0.04

# 99.9th Percentile

In [9]:
make_table_errors(concerned_Rs, concerned_rhos, 0.999)

R=1   R=3   R=6   R=9
$\rho = 0.2$  0.03  0.24  0.25  0.26
$\rho = 0.3$  0.06  0.25  0.34  0.35
$\rho = 0.4$  0.10  0.24  0.37  0.42
$\rho = 0.5$  0.08  0.22  0.38  0.44
$\rho = 0.6$  0.08  0.16  0.01  0.04
$\rho = 0.7$  0.08  0.09  0.08  0.04
$\rho = 0.8$  0.04  0.11  0.13  0.08

# 99.99th Percentile

In [10]:
make_table_errors(concerned_Rs, concerned_rhos, 0.9999)

R=1   R=3   R=6   R=9
$\rho = 0.2$  0.12  0.18  0.26  0.27
$\rho = 0.3$  0.01  0.24  0.32  0.37
$\rho = 0.4$  0.08  0.26  0.36  0.44
$\rho = 0.5$  0.06  0.13  0.40  0.45
$\rho = 0.6$  0.09  0.03  0.03  0.04
$\rho = 0.7$  0.04  0.19  0.10  0.06
$\rho = 0.8$  0.24  0.15  0.18  0.13

In [12]:
make_table_percentiles(concerned_Rs, concerned_rhos, 0.9999)

R=1    R=3    R=6    R=9
$\rho = 0.2$   15.79  10.12   9.35   9.23
$\rho = 0.3$   19.74  10.69   9.57   9.23
$\rho = 0.4$   23.40  11.69  10.07   9.38
$\rho = 0.5$   31.88  14.84  10.62   9.96
$\rho = 0.6$   42.42  19.22  12.85  10.80
$\rho = 0.7$   64.62  27.73  16.00  13.06
$\rho = 0.8$  104.95  41.73  23.84  17.62

# Get tables for best 99.99-percentile methods

Above we select the best distribution regarding Wasserstrein distance, and then measure the error in the 99.99-percentile sojourn time.

Below we get the best method for the 99.99 percentile, and measure in the 99.99-percentile sojourn time.

In [13]:
import pandas as pd
df = pd.read_csv('best-percentile-method.csv')

In [14]:
print('R,rho,del_9999')
table = []

for rho in [.1, .25, .5, .75, .85, .9, .95, .99]:
    row = []
    for R in [1,3,5,7,10]:
        del9999 = df[(df['percentile']==0.9999) & (df['R']==R) & (df['rho']==rho)]['abs_sojourn_err'].values[0]
        row.append(del9999)
    table.append(row)
    
pd.DataFrame(table)

R,rho,del_9999


0       1       2       3       4
0   -3.76    0.29    1.49    1.38    1.50
1   -0.68    2.38    2.58    2.94    3.04
2   -0.41    0.22    1.17    1.47    1.78
3   -6.56   -7.15   -4.51   -3.04   -1.87
4  -56.60   -8.83  -11.87   -9.84   -7.29
5 -109.06  -31.78  -24.69  -20.86  -16.63
6 -104.87  -89.57  -68.65  -54.94  -47.24
7 -180.58 -180.59 -180.60 -180.61 -180.62